# 营销归因 - 谁「真正」带来了转化？

---

## 欢迎来到营销归因的世界！

想象一下这个场景：

> 你是一家电商公司的营销总监。上个月，市场部在 5 个渠道投放了广告：
> - 💻 Google 搜索广告
> - 📱 朋友圈广告
> - 📧 EDM 邮件营销
> - 🎥 抖音短视频
> - 🛍️ 电商联盟
>
> 一个用户的购买路径是这样的：
> ```
> 朋友圈广告 → 搜索广告 → EDM → 搜索广告 → 购买！💰
> ```
>
> 现在 CFO 问你："这笔订单应该算谁的功劳？我们明年的预算要怎么分配？"

你可能会想：这还不简单？

- **销售说**："算最后一次搜索广告的！临门一脚最重要！"
- **品牌说**："算朋友圈广告的！第一印象才是转化的起点！"
- **增长说**："都重要，平均分配贡献！"
- **CFO 说**："我只关心谁带来了增量收入！"

每个人说的好像都有道理... 🤔

这就是**营销归因 (Marketing Attribution)** 要解决的问题！

---

## 📚 本节学习目标

1. 理解归因问题的本质（多触点归因 vs 单触点归因）
2. 掌握规则归因方法（Last-touch, First-touch, Linear, Time-decay, Position-based）
3. 理解 Shapley Value 归因的原理和计算
4. 掌握 Markov Chain 归因的建模
5. **区分触达归因 vs 增量归因**（核心！）
6. 将归因结果应用于预算优化决策

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations, permutations
from typing import List, Dict, Tuple, Set
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示和绘图风格
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

# 设置随机种子
np.random.seed(42)

print("✅ 环境准备完毕！让我们开始吧！")

---

## 🌟 Part 1: 归因问题的本质

### 什么是营销归因？

营销归因回答的核心问题是：

> **在一次转化（购买、注册等）中，多个营销触点各自的贡献是多少？**

### 一个真实的用户旅程

让我们看一个真实的例子（某电商用户的 30 天购买路径）：

```
Day 1:  朋友圈广告（首次接触）
Day 5:  Google 搜索广告（主动搜索）
Day 10: EDM 邮件（收到促销信息）
Day 15: Google 搜索广告（再次搜索）
Day 18: 抖音广告（看到种草视频）
Day 20: Google 搜索广告（最终购买）✅
```

这个路径中，用户在 20 天内接触了 **6 次营销触点**，涉及 **4 个渠道**。

### 归因 vs 因果推断

| 维度 | 归因 (Attribution) | 因果推断 (Causal Inference) |
|------|-------------------|----------------------------|
| 问题 | 已转化用户的触点如何"分功劳" | 渠道是否"导致"了转化 |
| 样本 | 只看转化用户 | 需要转化 + 未转化用户 |
| 反事实 | 不考虑 | 核心（如果没有这个渠道会怎样） |
| 实验需求 | 不需要 | 需要（或强假设） |
| 回答 | "参与了"转化 | "导致了"转化 |

**关键区别**：
- **归因**：基于已转化用户的历史触点数据，分配"功劳"
- **增量归因**：基于实验或准实验，估计渠道的"增量效应"

我们今天主要讨论前者（触达归因），最后会讨论如何做增量归因。

### 🔬 生成模拟数据

让我们生成一个模拟的用户路径数据集：

In [ ]:
def generate_user_journeys(n_users: int = 1000, seed: int = 42) -> pd.DataFrame:
    """
    生成用户转化路径数据
    
    每个用户可能经历多个营销触点，最终转化（或未转化）
    
    渠道：
    - Search: 搜索广告
    - Social: 社交媒体
    - Email: 邮件营销
    - Display: 展示广告
    - Affiliate: 电商联盟
    
    Returns:
        DataFrame with columns: user_id, path, converted, revenue
    """
    np.random.seed(seed)
    
    channels = ['Search', 'Social', 'Email', 'Display', 'Affiliate']
    
    # 常见路径模式（有些路径更容易导致转化）
    common_paths = [
        ['Social', 'Search'],                          # 品牌认知 → 主动搜索
        ['Search'],                                    # 直接搜索
        ['Social', 'Email', 'Search'],                 # 多次触达
        ['Display', 'Search'],                         # 展示 → 搜索
        ['Email', 'Search'],                           # 邮件 → 搜索
        ['Social', 'Search', 'Email', 'Search'],       # 复杂路径
        ['Affiliate', 'Search'],                       # 联盟 → 搜索
        ['Display', 'Social', 'Search'],               # 多渠道
    ]
    
    # 每种路径的转化概率（后面的路径更复杂，但转化率更高）
    conversion_probs = [0.05, 0.03, 0.15, 0.08, 0.10, 0.25, 0.12, 0.18]
    
    data = []
    
    for user_id in range(n_users):
        # 随机选择一个路径模式
        path_idx = np.random.choice(len(common_paths), p=[0.15, 0.20, 0.10, 0.12, 0.13, 0.08, 0.10, 0.12])
        path = common_paths[path_idx].copy()
        
        # 随机添加一些噪声（重复触点）
        if np.random.rand() < 0.3:
            path.append(np.random.choice(channels))
        
        # 判断是否转化
        converted = np.random.rand() < conversion_probs[path_idx]
        
        # 转化金额（如果转化）
        revenue = np.random.lognormal(4.5, 0.5) if converted else 0
        
        data.append({
            'user_id': user_id,
            'path': ' > '.join(path),
            'path_list': path,
            'converted': converted,
            'revenue': revenue,
            'touchpoints': len(path)
        })
    
    df = pd.DataFrame(data)
    
    # 只保留转化用户（归因只针对转化用户）
    df_converted = df[df['converted']].reset_index(drop=True)
    
    return df_converted

# 生成数据
df_journeys = generate_user_journeys(n_users=5000)

print(f"🎯 生成了 {len(df_journeys)} 个转化用户的路径数据")
print(f"\n数据预览:")
display(df_journeys.head(10))

# 统计信息
print(f"\n📊 基本统计:")
print(f"   总转化数: {len(df_journeys)}")
print(f"   总收入: ${df_journeys['revenue'].sum():,.2f}")
print(f"   平均客单价: ${df_journeys['revenue'].mean():.2f}")
print(f"   平均触点数: {df_journeys['touchpoints'].mean():.2f}")

In [ ]:
# 可视化路径长度分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 触点数分布
ax1 = axes[0]
df_journeys['touchpoints'].value_counts().sort_index().plot(kind='bar', ax=ax1, color='steelblue', edgecolor='black')
ax1.set_xlabel('路径长度（触点数）')
ax1.set_ylabel('用户数')
ax1.set_title('用户路径长度分布')
ax1.grid(axis='y', alpha=0.3)

# 渠道出现频次
ax2 = axes[1]
channel_counts = defaultdict(int)
for path_list in df_journeys['path_list']:
    for channel in path_list:
        channel_counts[channel] += 1

channel_df = pd.DataFrame(list(channel_counts.items()), columns=['Channel', 'Count']).sort_values('Count', ascending=True)
channel_df.plot(kind='barh', x='Channel', y='Count', ax=ax2, color='coral', edgecolor='black', legend=False)
ax2.set_xlabel('出现次数')
ax2.set_ylabel('渠道')
ax2.set_title('各渠道在转化路径中的出现频次')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

---

## 📊 Part 2: 规则归因方法

规则归因是最简单直接的方法，基于预定义的规则来分配功劳。

### 2.1 常见规则归因模型

| 模型 | 规则 | 适用场景 | 优点 | 缺点 |
|------|------|---------|------|------|
| **Last-touch** | 100% 归因给最后一个触点 | 短周期、冲动消费 | 简单；销售导向 | 忽略前期培育 |
| **First-touch** | 100% 归因给第一个触点 | 品牌营销 | 强调认知价值 | 忽略后续转化努力 |
| **Linear** | 平均分配 | 所有触点同等重要 | 公平；简单 | 不符合直觉 |
| **Time-decay** | 越接近转化权重越高 | 长周期决策 | 符合"临门一脚"直觉 | 参数敏感 |
| **Position-based** | 首末各 40%，中间 20% | 认知+转化都重要 | 平衡首末 | 中间触点被低估 |

### 2.2 数学定义

假设一个用户的转化路径为 $P = [c_1, c_2, ..., c_n]$，其中 $c_i$ 是第 $i$ 个触点。

**Last-touch Attribution:**
$$
w_i = \begin{cases}
1 & i = n \\
0 & i \neq n
\end{cases}
$$

**First-touch Attribution:**
$$
w_i = \begin{cases}
1 & i = 1 \\
0 & i \neq 1
\end{cases}
$$

**Linear Attribution:**
$$
w_i = \frac{1}{n} \quad \forall i
$$

**Time-decay Attribution:**
$$
w_i = \frac{e^{-\lambda(n-i)}}{\sum_{j=1}^{n} e^{-\lambda(n-j)}}
$$
其中 $\lambda$ 是衰减率参数（通常取 0.5-2）。

**Position-based Attribution (40-20-40):**
$$
w_i = \begin{cases}
0.4 & i = 1 \\
0.4 & i = n \\
\frac{0.2}{n-2} & 1 < i < n
\end{cases}
$$

In [ ]:
class RuleBasedAttribution:
    """
    规则归因模型
    """
    
    @staticmethod
    def last_touch(path: List[str]) -> Dict[str, float]:
        """
        Last-touch 归因：100% 归给最后一个触点
        """
        attribution = {}
        for channel in set(path):
            attribution[channel] = 1.0 if channel == path[-1] else 0.0
        return attribution
    
    @staticmethod
    def first_touch(path: List[str]) -> Dict[str, float]:
        """
        First-touch 归因：100% 归给第一个触点
        """
        attribution = {}
        for channel in set(path):
            attribution[channel] = 1.0 if channel == path[0] else 0.0
        return attribution
    
    @staticmethod
    def linear(path: List[str]) -> Dict[str, float]:
        """
        Linear 归因：平均分配权重
        """
        attribution = defaultdict(float)
        weight_per_touch = 1.0 / len(path)
        for channel in path:
            attribution[channel] += weight_per_touch
        return dict(attribution)
    
    @staticmethod
    def time_decay(path: List[str], decay_rate: float = 0.7) -> Dict[str, float]:
        """
        Time-decay 归因：越接近转化，权重越高
        
        Args:
            decay_rate: 衰减率，越大则最后触点权重越大
        """
        n = len(path)
        attribution = defaultdict(float)
        
        weights = []
        for i in range(n):
            weight = np.exp(-decay_rate * (n - 1 - i))
            weights.append(weight)
        
        # 归一化
        total = sum(weights)
        weights = [w / total for w in weights]
        
        # 累加同一渠道的权重
        for i, channel in enumerate(path):
            attribution[channel] += weights[i]
        
        return dict(attribution)
    
    @staticmethod
    def position_based(path: List[str], first_weight: float = 0.4, last_weight: float = 0.4) -> Dict[str, float]:
        """
        Position-based 归因：首末各占固定比例，中间平分剩余
        
        Args:
            first_weight: 第一个触点的权重（默认 0.4）
            last_weight: 最后一个触点的权重（默认 0.4）
        """
        n = len(path)
        attribution = defaultdict(float)
        
        if n == 1:
            attribution[path[0]] = 1.0
        elif n == 2:
            attribution[path[0]] = 0.5
            attribution[path[1]] = 0.5
        else:
            # 第一个触点
            attribution[path[0]] += first_weight
            # 最后一个触点
            attribution[path[-1]] += last_weight
            # 中间触点平分剩余权重
            middle_weight = (1.0 - first_weight - last_weight) / (n - 2)
            for i in range(1, n - 1):
                attribution[path[i]] += middle_weight
        
        return dict(attribution)

# 测试
test_path = ['Social', 'Search', 'Email', 'Search']
print("🧪 测试路径:", ' > '.join(test_path))
print("\n各种归因结果:")
print(f"Last-touch:     {RuleBasedAttribution.last_touch(test_path)}")
print(f"First-touch:    {RuleBasedAttribution.first_touch(test_path)}")
print(f"Linear:         {RuleBasedAttribution.linear(test_path)}")
print(f"Time-decay:     {RuleBasedAttribution.time_decay(test_path)}")
print(f"Position-based: {RuleBasedAttribution.position_based(test_path)}")

In [ ]:
def apply_attribution(df: pd.DataFrame, method: str = 'last_touch') -> pd.DataFrame:
    """
    对所有用户路径应用归因方法
    
    Returns:
        DataFrame with channel-level attribution results
    """
    channel_attribution = defaultdict(float)
    channel_revenue = defaultdict(float)
    channel_conversions = defaultdict(int)
    
    for idx, row in df.iterrows():
        path = row['path_list']
        revenue = row['revenue']
        
        # 选择归因方法
        if method == 'last_touch':
            weights = RuleBasedAttribution.last_touch(path)
        elif method == 'first_touch':
            weights = RuleBasedAttribution.first_touch(path)
        elif method == 'linear':
            weights = RuleBasedAttribution.linear(path)
        elif method == 'time_decay':
            weights = RuleBasedAttribution.time_decay(path)
        elif method == 'position_based':
            weights = RuleBasedAttribution.position_based(path)
        else:
            raise ValueError(f"Unknown method: {method}")
        
        # 分配收入
        for channel, weight in weights.items():
            channel_revenue[channel] += revenue * weight
            channel_attribution[channel] += weight
            if weight > 0:
                channel_conversions[channel] += 1
    
    # 创建结果 DataFrame
    result_df = pd.DataFrame({
        'Channel': list(channel_revenue.keys()),
        'Attributed_Revenue': list(channel_revenue.values()),
        'Attributed_Conversions': [channel_attribution[ch] for ch in channel_revenue.keys()],
        'Touch_Count': [channel_conversions[ch] for ch in channel_revenue.keys()]
    }).sort_values('Attributed_Revenue', ascending=False)
    
    result_df['Revenue_Share'] = result_df['Attributed_Revenue'] / result_df['Attributed_Revenue'].sum()
    
    return result_df

# 对比不同归因方法
methods = ['last_touch', 'first_touch', 'linear', 'time_decay', 'position_based']
results = {}

for method in methods:
    results[method] = apply_attribution(df_journeys, method)

# 可视化对比
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, method in enumerate(methods):
    ax = axes[idx]
    data = results[method].sort_values('Attributed_Revenue', ascending=True)
    data.plot(kind='barh', x='Channel', y='Attributed_Revenue', ax=ax, color='skyblue', edgecolor='black', legend=False)
    ax.set_xlabel('归因收入 ($)')
    ax.set_ylabel('')
    ax.set_title(f'{method.replace("_", "-").title()} Attribution')
    ax.grid(axis='x', alpha=0.3)
    
    # 添加百分比标签
    for i, (channel, revenue) in enumerate(zip(data['Channel'], data['Attributed_Revenue'])):
        share = revenue / data['Attributed_Revenue'].sum() * 100
        ax.text(revenue, i, f' {share:.1f}%', va='center', fontsize=9)

# 移除多余的子图
axes[-1].axis('off')

plt.tight_layout()
plt.show()

print("\n📊 不同归因方法的对比（收入占比）:")
comparison_df = pd.DataFrame({
    method: results[method].set_index('Channel')['Revenue_Share'] * 100
    for method in methods
})
display(comparison_df.round(1))

### 🤔 规则归因的问题

从上面的结果可以看出，**不同的归因方法会得到截然不同的结论**！

- **Last-touch**: Search 占比最高（因为经常是最后触点）
- **First-touch**: Social 占比提升（因为常作为首次接触）
- **Linear**: 相对平均

**核心问题**：规则归因是**主观的、武断的**，缺乏理论基础。

那有没有更"公平"、更有理论基础的方法呢？

有！**Shapley Value** 归因！

---

## 🎲 Part 3: Shapley Value 归因

### 3.1 合作博弈论基础

**Shapley Value** 来自合作博弈论，由 Lloyd Shapley 于 1953 年提出（他因此获得了 2012 年诺贝尔经济学奖）。

#### 一个生动的例子：三个人合作开店

假设有三个人合作开了一家奶茶店：
- **Alice**：提供资金 💰
- **Bob**：提供场地 🏠
- **Carol**：提供技术（会做奶茶）🧋

店铺最终赚了 **100 万**。问题：如何"公平"地分钱？

我们可以计算每种组合的收益：

| 组合 | 能开店吗？ | 收益 |
|------|-----------|------|
| {} | ❌ | 0 |
| {Alice} | ❌（没场地和技术） | 0 |
| {Bob} | ❌（没钱和技术） | 0 |
| {Carol} | ❌（没钱和场地） | 0 |
| {Alice, Bob} | ❌（没人会做） | 0 |
| {Alice, Carol} | 🔺（可以租场地）| 50 万 |
| {Bob, Carol} | 🔺（可以借钱）| 60 万 |
| {Alice, Bob, Carol} | ✅ | 100 万 |

**Shapley Value 的思想**：
> 一个人的贡献 = 平均而言，TA 加入后带来的增量收益

### 3.2 Shapley Value 的定义

对于合作博弈 $(N, v)$，其中：
- $N$ 是玩家集合（在营销归因中是渠道集合）
- $v(S)$ 是子集 $S$ 的价值函数（转化价值）

玩家 $i$ 的 Shapley Value 定义为：

$$
\phi_i(v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|! \cdot (|N| - |S| - 1)!}{|N|!} \cdot [v(S \cup \{i\}) - v(S)]
$$

**人话翻译**：
- 遍历所有不包含 $i$ 的子集 $S$
- 计算 $i$ 加入 $S$ 后的增量：$v(S \cup \{i\}) - v(S)$
- 对所有可能的加入顺序取平均（权重是 $\frac{|S|! \cdot (|N| - |S| - 1)!}{|N|!}$）

### 3.3 Shapley Value 的公理性

Shapley Value 是**唯一**满足以下公理的分配方案：

1. **效率性 (Efficiency)**: $\sum_{i \in N} \phi_i(v) = v(N)$ （所有价值被完全分配）
2. **对称性 (Symmetry)**: 如果 $i, j$ 可互换，则 $\phi_i(v) = \phi_j(v)$
3. **虚拟性 (Null player)**: 如果 $v(S \cup \{i\}) = v(S)$ 对所有 $S$ 成立，则 $\phi_i(v) = 0$
4. **可加性 (Additivity)**: $\phi_i(v + w) = \phi_i(v) + \phi_i(w)$

这些公理保证了 Shapley Value 的"公平性"。

### 3.4 在营销归因中的应用

将用户转化路径视为合作博弈：
- **玩家**: 路径中的各个渠道
- **价值函数** $v(S)$: 包含子集 $S$ 中渠道的路径的平均转化率（或收入）

例如，路径 `Social > Search > Email > Search`：
- $N = \{\text{Social}, \text{Search}, \text{Email}\}$
- $v(\{\text{Search}\})$ = 包含 Search 的路径的平均转化价值
- $v(\{\text{Social}, \text{Search}\})$ = 包含 Social 和 Search 的路径的平均转化价值

In [ ]:
class ShapleyAttribution:
    """
    Shapley Value 归因
    """
    
    def __init__(self, df: pd.DataFrame):
        """
        Args:
            df: 包含 'path_list' 和 'revenue' 列的 DataFrame
        """
        self.df = df
        self.all_channels = set()
        for path in df['path_list']:
            self.all_channels.update(path)
        
        # 构建价值函数：每个渠道子集对应的平均收入
        self.value_function = self._build_value_function()
    
    def _build_value_function(self) -> Dict[frozenset, float]:
        """
        构建价值函数 v(S)
        
        v(S) = 包含子集 S 中所有渠道的路径的平均收入
        """
        value_func = {}
        
        # 枚举所有可能的渠道子集
        from itertools import chain, combinations
        
        def powerset(iterable):
            "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
            s = list(iterable)
            return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
        
        for subset in powerset(self.all_channels):
            subset_set = frozenset(subset)
            
            if len(subset_set) == 0:
                value_func[subset_set] = 0.0
                continue
            
            # 找到包含这个子集所有渠道的路径
            matching_revenues = []
            for idx, row in self.df.iterrows():
                path_channels = set(row['path_list'])
                if subset_set.issubset(path_channels):
                    matching_revenues.append(row['revenue'])
            
            # 计算平均收入（如果没有匹配的路径，值为 0）
            value_func[subset_set] = np.mean(matching_revenues) if matching_revenues else 0.0
        
        return value_func
    
    def compute_shapley(self, channel: str) -> float:
        """
        计算单个渠道的 Shapley Value
        
        公式: φ_i = Σ_{S ⊆ N\{i}} [|S|!(|N|-|S|-1)! / |N|!] * [v(S∪{i}) - v(S)]
        """
        n = len(self.all_channels)
        other_channels = self.all_channels - {channel}
        
        shapley_value = 0.0
        
        # 枚举所有不包含当前渠道的子集
        from itertools import chain, combinations
        
        def powerset(iterable):
            s = list(iterable)
            return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
        
        for subset in powerset(other_channels):
            subset_set = frozenset(subset)
            s_size = len(subset_set)
            
            # 计算权重: |S|! * (|N| - |S| - 1)! / |N|!
            from math import factorial
            weight = factorial(s_size) * factorial(n - s_size - 1) / factorial(n)
            
            # 计算边际贡献: v(S ∪ {i}) - v(S)
            v_with = self.value_function.get(subset_set | {channel}, 0.0)
            v_without = self.value_function.get(subset_set, 0.0)
            marginal_contribution = v_with - v_without
            
            shapley_value += weight * marginal_contribution
        
        return shapley_value
    
    def compute_all_shapley(self) -> Dict[str, float]:
        """
        计算所有渠道的 Shapley Value
        """
        shapley_values = {}
        for channel in self.all_channels:
            shapley_values[channel] = self.compute_shapley(channel)
        return shapley_values

# 计算 Shapley 归因
print("🎲 计算 Shapley Value 归因...（可能需要几秒钟）")
shapley_model = ShapleyAttribution(df_journeys)
shapley_values = shapley_model.compute_all_shapley()

print("\n✅ Shapley Value 归因结果:")
shapley_df = pd.DataFrame(list(shapley_values.items()), columns=['Channel', 'Shapley_Value']).sort_values('Shapley_Value', ascending=False)
shapley_df['Value_Share'] = shapley_df['Shapley_Value'] / shapley_df['Shapley_Value'].sum()
display(shapley_df)

# 将 Shapley 转换为收入归因
total_revenue = df_journeys['revenue'].sum()
shapley_df['Attributed_Revenue'] = shapley_df['Shapley_Value'] / shapley_df['Shapley_Value'].sum() * total_revenue

print(f"\n总收入: ${total_revenue:,.2f}")
print(f"Shapley 归因总和: ${shapley_df['Attributed_Revenue'].sum():,.2f} (应该相等)")

In [ ]:
# 对比 Shapley 与规则归因
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

all_methods = methods + ['shapley']
all_results = results.copy()
all_results['shapley'] = shapley_df.rename(columns={'Shapley_Value': 'Attributed_Revenue'})[['Channel', 'Attributed_Revenue']]

for idx, method in enumerate(all_methods):
    ax = axes[idx]
    data = all_results[method].sort_values('Attributed_Revenue', ascending=True)
    
    color = 'gold' if method == 'shapley' else 'skyblue'
    data.plot(kind='barh', x='Channel', y='Attributed_Revenue', ax=ax, color=color, edgecolor='black', legend=False)
    ax.set_xlabel('归因值')
    ax.set_ylabel('')
    
    title = 'Shapley Value ⭐' if method == 'shapley' else f'{method.replace("_", "-").title()}'
    ax.set_title(title, fontweight='bold' if method == 'shapley' else 'normal')
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# 数值对比
print("\n📊 所有方法的对比（收入占比 %）:")
comparison_df = pd.DataFrame()
for method in all_methods:
    df_method = all_results[method].copy()
    df_method['Share'] = df_method['Attributed_Revenue'] / df_method['Attributed_Revenue'].sum() * 100
    comparison_df[method] = df_method.set_index('Channel')['Share']

display(comparison_df.round(1))

### 🎯 Shapley Value 的优势

1. **理论基础**: 基于合作博弈论，有严格的公理化基础
2. **公平性**: 唯一满足 4 个公理的分配方案
3. **考虑交互**: 考虑了渠道之间的协同效应

### ⚠️ Shapley Value 的局限

1. **计算复杂度**: $O(2^n)$ 指数复杂度，渠道多时很慢
2. **数据需求**: 需要足够的数据来估计所有子集的价值
3. **仍是触达归因**: 没有考虑反事实（"如果没有这个渠道会怎样"）

---

## 🔗 Part 4: Markov Chain 归因

### 4.1 马尔可夫链的思想

**核心思想**: 将用户旅程建模为马尔可夫链，渠道的价值 = 移除该渠道后转化率的下降。

### 4.2 建模步骤

1. **构建转移概率矩阵**: 从一个渠道到另一个渠道的概率
2. **计算转化概率**: 从起点到转化的概率
3. **移除效应**: 移除某个渠道后，转化概率下降多少

### 4.3 转移概率矩阵

$$
P_{ij} = P(\text{下一个状态是 } j | \text{当前状态是 } i)
$$

例如：
```
         Social  Search  Email  Conversion
Social     0      0.7    0.2      0.1
Search    0.1     0.3    0.4      0.2
Email     0.2     0.5     0       0.3
```

### 4.4 移除效应

渠道 $c$ 的贡献：

$$
\text{Removal Effect}_c = \frac{P(\text{Conversion}) - P(\text{Conversion | remove } c)}{P(\text{Conversion})}
$$

In [ ]:
class MarkovAttribution:
    """
    Markov Chain 归因模型
    """
    
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.channels = set()
        for path in df['path_list']:
            self.channels.update(path)
        
        # 构建转移矩阵
        self.transition_matrix = self._build_transition_matrix()
    
    def _build_transition_matrix(self, excluded_channel: str = None) -> pd.DataFrame:
        """
        构建转移概率矩阵
        
        Args:
            excluded_channel: 要排除的渠道（用于计算移除效应）
        """
        # 统计转移次数
        transitions = defaultdict(lambda: defaultdict(int))
        
        for idx, row in self.df.iterrows():
            path = row['path_list']
            
            # 过滤掉被排除的渠道
            if excluded_channel:
                path = [ch for ch in path if ch != excluded_channel]
            
            if len(path) == 0:
                continue
            
            # Start -> 第一个渠道
            transitions['Start'][path[0]] += 1
            
            # 渠道之间的转移
            for i in range(len(path) - 1):
                transitions[path[i]][path[i+1]] += 1
            
            # 最后一个渠道 -> Conversion
            transitions[path[-1]]['Conversion'] += 1
        
        # 转换为概率（归一化）
        transition_probs = {}
        for from_state, to_states in transitions.items():
            total = sum(to_states.values())
            transition_probs[from_state] = {to_state: count / total for to_state, count in to_states.items()}
        
        return transition_probs
    
    def _conversion_probability(self, transition_matrix: dict, max_steps: int = 20) -> float:
        """
        计算从 Start 到 Conversion 的概率（使用蒙特卡洛模拟）
        """
        n_simulations = 10000
        conversions = 0
        
        for _ in range(n_simulations):
            state = 'Start'
            for step in range(max_steps):
                if state == 'Conversion':
                    conversions += 1
                    break
                
                if state not in transition_matrix or len(transition_matrix[state]) == 0:
                    break
                
                # 根据转移概率选择下一个状态
                next_states = list(transition_matrix[state].keys())
                probs = list(transition_matrix[state].values())
                state = np.random.choice(next_states, p=probs)
        
        return conversions / n_simulations
    
    def compute_removal_effects(self) -> Dict[str, float]:
        """
        计算每个渠道的移除效应
        """
        # 基准转化率（所有渠道都在）
        base_conversion_prob = self._conversion_probability(self.transition_matrix)
        
        removal_effects = {}
        
        for channel in self.channels:
            # 移除该渠道后重建转移矩阵
            transition_without = self._build_transition_matrix(excluded_channel=channel)
            
            # 计算移除后的转化率
            conversion_without = self._conversion_probability(transition_without)
            
            # 移除效应 = 转化率的下降比例
            if base_conversion_prob > 0:
                removal_effect = (base_conversion_prob - conversion_without) / base_conversion_prob
            else:
                removal_effect = 0.0
            
            removal_effects[channel] = max(removal_effect, 0.0)  # 避免负值
        
        return removal_effects

# 计算 Markov 归因
print("🔗 计算 Markov Chain 归因...（需要运行蒙特卡洛模拟）")
markov_model = MarkovAttribution(df_journeys)
removal_effects = markov_model.compute_removal_effects()

# 归一化为归因权重
total_removal = sum(removal_effects.values())
markov_attribution = {ch: effect / total_removal for ch, effect in removal_effects.items()}

print("\n✅ Markov Chain 归因结果:")
markov_df = pd.DataFrame(list(markov_attribution.items()), columns=['Channel', 'Markov_Attribution']).sort_values('Markov_Attribution', ascending=False)
markov_df['Attributed_Revenue'] = markov_df['Markov_Attribution'] * total_revenue
display(markov_df)

In [ ]:
# 最终对比：所有方法
final_comparison = pd.DataFrame()

for method in ['last_touch', 'shapley']:
    df_method = all_results[method].copy()
    df_method['Share'] = df_method['Attributed_Revenue'] / df_method['Attributed_Revenue'].sum() * 100
    final_comparison[method] = df_method.set_index('Channel')['Share']

# 添加 Markov
final_comparison['markov'] = markov_df.set_index('Channel')['Markov_Attribution'] * 100

# 可视化
fig, ax = plt.subplots(figsize=(10, 6))
final_comparison.plot(kind='bar', ax=ax, width=0.8, edgecolor='black')
ax.set_ylabel('归因占比 (%)')
ax.set_xlabel('渠道')
ax.set_title('三种归因方法的对比', fontsize=14, fontweight='bold')
ax.legend(['Last-touch', 'Shapley Value', 'Markov Chain'])
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n📊 最终对比（归因占比 %）:")
display(final_comparison.round(1))

---

## ⚡ Part 5: 增量归因 vs 触达归因

### 5.1 本质区别

前面所有的方法（规则归因、Shapley、Markov）都是**触达归因 (Touch Attribution)**：

| 维度 | 触达归因 | 增量归因 |
|------|---------|----------|
| **问题** | 谁"参与了"转化？ | 谁"导致了"转化？ |
| **数据** | 只看转化用户的路径 | 需要转化 + 未转化用户 |
| **反事实** | ❌ 不考虑 | ✅ 核心 |
| **方法** | 规则、Shapley、Markov | RCT、PSM、DID 等 |
| **回答** | "在转化路径中，谁出现了" | "如果没有这个渠道，会转化吗" |

### 5.2 一个重要的例子

假设有两个渠道：
- **Brand Display**（品牌展示广告）：覆盖所有人，但转化率低
- **Retargeting**（重定向广告）：只针对已访问用户，转化率高

**触达归因**会发现：
- Retargeting 在大部分转化路径的最后
- → Last-touch 会给 Retargeting 很高的权重

**但实际情况可能是**：
- Retargeting 只是"锦上添花"，这些用户本来就会转化
- Brand Display 才是"雪中送炭"，带来了新用户

**增量归因**会通过实验发现：
- Retargeting 的增量效应：+2%
- Brand Display 的增量效应：+15%

### 5.3 如何做增量归因？

**方法 1: 随机实验 (RCT)**
- 随机分为实验组（投放渠道 A）和对照组（不投放）
- 比较两组的转化率差异 → 渠道 A 的增量效应
- **金标准**，但成本高、周期长

**方法 2: Geo-experiment**
- 在不同地区随机开关渠道
- 适合无法做用户级随机化的场景（如电视广告）

**方法 3: PSM / DML**
- 如果无法做实验，使用观测数据的因果推断方法
- 比较"看到广告的用户" vs "倾向性得分匹配的未看到广告的用户"

**方法 4: Ghost Ads**
- 在广告竞价中，随机"放弃"一部分曝光机会
- 比较"赢得曝光" vs "放弃曝光"的用户的转化率
- Facebook、Google 都在用

### 5.4 简单模拟：增量 vs 触达

让我们用一个简化的例子说明差异：

In [ ]:
def simulate_incremental_vs_touch():
    """
    模拟增量归因 vs 触达归因的差异
    
    场景：
    - Channel A (Brand): 覆盖所有人，带来认知，增量效应 +10%
    - Channel B (Retargeting): 只针对高意向用户，出现在最后，增量效应 +2%
    """
    np.random.seed(42)
    n_users = 10000
    
    # 用户的基础转化倾向（0-1）
    base_propensity = np.random.beta(2, 8, n_users)  # 大部分用户基础转化率低
    
    # Channel A: 覆盖所有人
    exposed_to_A = np.ones(n_users, dtype=bool)
    
    # Channel B: 只针对高倾向用户（top 30%）
    threshold = np.percentile(base_propensity, 70)
    exposed_to_B = base_propensity >= threshold
    
    # 真实的增量效应
    # Channel A: 让转化概率 +10%
    # Channel B: 让转化概率 +2%
    conversion_prob = base_propensity.copy()
    conversion_prob += 0.10 * exposed_to_A  # Channel A 的增量
    conversion_prob += 0.02 * exposed_to_B  # Channel B 的增量
    conversion_prob = np.clip(conversion_prob, 0, 1)
    
    # 实际转化
    converted = np.random.rand(n_users) < conversion_prob
    
    # ========== 增量归因（真实效应）==========
    # 这是我们在模拟中设定的真实值
    true_incremental = {
        'Channel_A': 0.10,
        'Channel_B': 0.02
    }
    
    # ========== 触达归因（Last-touch）==========
    # 只看转化用户的路径
    converted_users = converted
    
    # 构造路径：A -> B（如果暴露于 B）或者只有 A
    paths = []
    for i in range(n_users):
        if not converted[i]:
            continue
        
        if exposed_to_B[i]:
            paths.append(['Channel_A', 'Channel_B'])
        else:
            paths.append(['Channel_A'])
    
    # Last-touch 归因
    last_touch_counts = Counter([path[-1] for path in paths])
    total_conversions = sum(last_touch_counts.values())
    last_touch_attribution = {ch: count / total_conversions for ch, count in last_touch_counts.items()}
    
    # 结果
    print("🔬 增量归因 vs 触达归因对比\n")
    print("=" * 60)
    print(f"{'渠道':<15} {'真实增量效应':<20} {'Last-touch 归因':<20}")
    print("=" * 60)
    
    for channel in ['Channel_A', 'Channel_B']:
        incremental = true_incremental.get(channel, 0) * 100
        touch = last_touch_attribution.get(channel, 0) * 100
        print(f"{channel:<15} {incremental:>6.1f}% (真实价值) {touch:>12.1f}% (归因结果)")
    
    print("=" * 60)
    print("\n⚠️ 观察:")
    print("   - Channel A 的真实增量效应是 10%，但 Last-touch 只给了它很少的权重")
    print("   - Channel B 的真实增量效应只有 2%，但因为它总是最后触点，得到了大部分归因")
    print("   - 这就是触达归因的问题：混淆了'参与'和'导致'")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    
    channels = ['Channel A\n(Brand)', 'Channel B\n(Retargeting)']
    incremental_values = [10, 2]
    touch_values = [last_touch_attribution.get('Channel_A', 0) * 100, 
                    last_touch_attribution.get('Channel_B', 0) * 100]
    
    x = np.arange(len(channels))
    width = 0.35
    
    ax.bar(x - width/2, incremental_values, width, label='真实增量效应', color='green', edgecolor='black', alpha=0.8)
    ax.bar(x + width/2, touch_values, width, label='Last-touch 归因', color='orange', edgecolor='black', alpha=0.8)
    
    ax.set_ylabel('百分比 (%)', fontsize=12)
    ax.set_title('增量归因 vs 触达归因的差异', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(channels)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # 添加数值标签
    for i, (inc, touch) in enumerate(zip(incremental_values, touch_values)):
        ax.text(i - width/2, inc + 1, f'{inc:.1f}%', ha='center', fontsize=10, fontweight='bold')
        ax.text(i + width/2, touch + 1, f'{touch:.1f}%', ha='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

simulate_incremental_vs_touch()

### 🎓 关键启示

1. **触达归因告诉你"谁参与了"，增量归因告诉你"谁导致了"**
2. **触达归因会高估"最后触点"的价值**（因为高意向用户更容易被 retargeting）
3. **触达归因会低估"上游渠道"的价值**（如品牌广告）
4. **业务决策应基于增量归因**（否则会错误分配预算）

### 💡 实践建议

| 场景 | 用哪种归因？ |
|------|-------------|
| 快速了解用户路径 | 触达归因 |
| 预算分配决策 | 增量归因 |
| 渠道效果评估 | 增量归因 |
| 无法做实验 | 触达归因 + 业务判断 |
| 有实验能力 | 增量归因（RCT） |

---

## 💼 Part 6: 业务实践

### 6.1 案例：多渠道营销归因

假设你是一家 SaaS 公司的增长负责人，现在要决定明年的营销预算分配。

**现状**：
- 5 个渠道，总预算 100 万
- 今年的分配：Search 30万、Social 25万、Email 20万、Display 15万、Affiliate 10万

**数据**：
- 触达归因结果（基于历史转化路径）
- 增量实验结果（在部分地区做的 Geo-experiment）

**问题**：如何调整预算？

In [ ]:
# 模拟业务场景
budget_data = pd.DataFrame({
    'Channel': ['Search', 'Social', 'Email', 'Display', 'Affiliate'],
    'Current_Budget': [300000, 250000, 200000, 150000, 100000],
    'Last_Touch_Attribution_%': [45, 15, 10, 20, 10],
    'Shapley_Attribution_%': [30, 20, 15, 25, 10],
    'Incremental_Lift_%': [5, 15, 8, 12, 3],  # 来自实验
    'Cost_per_Conversion': [50, 80, 30, 100, 40]
})

# 计算 ROI
budget_data['Incremental_ROI'] = budget_data['Incremental_Lift_%'] / budget_data['Cost_per_Conversion']

print("📊 渠道分析报告\n")
display(budget_data)

# 对比不同归因方法的建议
print("\n💡 不同归因方法的预算建议:\n")

print("1️⃣ 基于 Last-touch 归因:")
print("   → 增加 Search 预算（占比 45%）")
print("   → 减少 Email 预算（占比 10%）")

print("\n2️⃣ 基于 Shapley 归因:")
print("   → 更平衡，Display 和 Social 应获得更多预算")

print("\n3️⃣ 基于增量归因 (推荐):")
print("   → Social 增量最高（15%），应增加预算")
print("   → Affiliate 增量最低（3%），考虑减少")
print("   → 结合 ROI，Email 性价比最高")

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 子图 1: 归因对比
ax1 = axes[0]
budget_data_sorted = budget_data.sort_values('Channel')
x = np.arange(len(budget_data_sorted))
width = 0.25

ax1.bar(x - width, budget_data_sorted['Last_Touch_Attribution_%'], width, label='Last-touch', alpha=0.8)
ax1.bar(x, budget_data_sorted['Shapley_Attribution_%'], width, label='Shapley', alpha=0.8)
ax1.bar(x + width, budget_data_sorted['Incremental_Lift_%'], width, label='Incremental', alpha=0.8)

ax1.set_ylabel('百分比 (%)')
ax1.set_title('不同归因方法的结果对比')
ax1.set_xticks(x)
ax1.set_xticklabels(budget_data_sorted['Channel'], rotation=45, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 子图 2: 增量 ROI
ax2 = axes[1]
budget_data_sorted_roi = budget_data.sort_values('Incremental_ROI', ascending=True)
budget_data_sorted_roi.plot(kind='barh', x='Channel', y='Incremental_ROI', ax=ax2, color='coral', edgecolor='black', legend=False)
ax2.set_xlabel('增量 ROI (Lift % / Cost)')
ax2.set_ylabel('')
ax2.set_title('各渠道的增量 ROI')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 6.2 常见陷阱

#### 陷阱 1: 过度依赖 Last-touch
**问题**: 会导致过度投资下游渠道（如 Retargeting），忽视品牌建设  
**后果**: 短期 ROI 看起来好，长期增长乏力

#### 陷阱 2: 混淆相关与因果
**问题**: 高意向用户更容易看到 Retargeting 广告  
**后果**: 误以为 Retargeting 效果好，实际只是"顺水推舟"

#### 陷阱 3: 忽视渠道协同
**问题**: 单独评估每个渠道，忽视组合效应  
**后果**: 可能错误地关闭某个"辅助"渠道，导致整体效果下降

#### 陷阱 4: 数据偏差
**问题**: Cookie 丢失、跨设备追踪不全  
**后果**: 归因数据本身就不准确

### 6.3 最佳实践

1. **用触达归因了解用户旅程，用增量归因做决策**
2. **定期做增量实验（至少每季度一次）**
3. **关注 Holdout 组（不投放任何广告的对照组）**
4. **警惕"归因通货膨胀"**（所有渠道的归因加起来 > 100%）
5. **结合业务常识**（数据是工具，不是答案）

---

## 🧠 练习与思考题

### 练习 1: 实现自己的归因模型

请实现一个自定义的 Time-decay 归因，衰减参数可调。

In [ ]:
def custom_time_decay(path: List[str], half_life_days: int = 7) -> Dict[str, float]:
    """
    实现一个基于半衰期的 Time-decay 归因
    
    假设触点之间间隔 1 天，权重按半衰期衰减。
    
    Args:
        path: 转化路径
        half_life_days: 半衰期（天）
    
    Returns:
        各渠道的归因权重
    
    提示: 
    - 权重 w_i = 2^(-(n-i)/half_life)
    - 记得归一化
    """
    n = len(path)
    attribution = defaultdict(float)
    
    # 计算每个触点的权重
    weights = []
    for i in range(n):
        # 距离转化的天数（第0个触点距离转化n-1天）
        days_to_conversion = n - 1 - i
        # 半衰期衰减：w = 2^(-days/half_life)
        weight = 2 ** (-days_to_conversion / half_life_days)
        weights.append(weight)
    
    # 归一化
    total_weight = sum(weights)
    weights = [w / total_weight for w in weights]
    
    # 累加同一渠道的权重
    for i, channel in enumerate(path):
        attribution[channel] += weights[i]
    
    return dict(attribution)

# 测试
test_path = ['Social', 'Search', 'Email', 'Search']
result = custom_time_decay(test_path, half_life_days=7)
print(f"自定义 Time-decay 结果: {result}")

# 验证总和为1
print(f"权重总和: {sum(result.values()):.4f} (应该为 1.0)")

### 练习 2: Shapley 值的性质验证

验证 Shapley 值的"效率性"公理：所有渠道的 Shapley 值之和应该等于总价值。

In [ ]:
# 验证 Shapley 值的"效率性"公理

# Shapley 值的效率性公理：所有渠道的 Shapley 值之和应该等于总价值
print("🔍 验证 Shapley Value 的效率性公理")
print("="*60)

# 计算所有 Shapley 值的总和
shapley_sum = sum(shapley_values.values())
print(f"\nShapley 值总和: {shapley_sum:.4f}")

# 计算总价值（所有渠道都存在时的转化价值）
all_channels_frozenset = frozenset(shapley_model.all_channels)
total_value = shapley_model.value_function.get(all_channels_frozenset, 0.0)
print(f"总价值 v(N): {total_value:.4f}")

# 检查是否相等
difference = abs(shapley_sum - total_value)
is_efficient = difference < 0.01  # 允许微小的数值误差

print(f"\n差异: {difference:.6f}")
print(f"效率性公理满足: {'✅ 是' if is_efficient else '❌ 否'}")

print(f"\n💡 解释:")
print(f"  根据 Shapley 值的效率性公理，所有玩家的 Shapley 值之和")
print(f"  应该等于大联盟的总价值。")
print(f"  ")
print(f"  公式: Σ φ_i(v) = v(N)")
print(f"  ")
print(f"  这意味着：Shapley 归因会将总收益\"公平\"地完全分配给各个渠道，")
print(f"  不会多分也不会少分。")

<cell_type>markdown</cell_type>---

## 🧠 练习与思考题

### 练习 1: 实现自己的归因模型

请实现一个自定义的 Time-decay 归因，衰减参数可调。

---

## 📝 思考题参考答案

### 问题 1: Last-touch 归因有什么根本问题？

**参考答案**:

Last-touch 归因的根本问题是**混淆了"参与"和"导致"**，具体包括：

1. **忽略前期培育**:
   - 用户可能因为品牌广告产生认知，但归因全给最后的搜索广告
   - 例如：用户看到朋友圈广告 → 产生兴趣 → 1周后主动搜索 → 购买
   - Last-touch 会100%归因给搜索，忽略朋友圈广告的"种草"作用

2. **Selection Bias（选择偏差）**:
   - 高意向用户更容易被 Retargeting 捕获
   - Retargeting 总是出现在最后，但可能只是"顺水推舟"
   - 这些用户即使不看 Retargeting 也会购买（Sure Things）

3. **鼓励短视行为**:
   - 营销团队会过度投资"临门一脚"的渠道
   - 忽视长期品牌建设和上游渠道

4. **数学上的不合理性**:
   - 如果路径是 A → B → C，Last-touch 给 C 全部权重
   - 但如果路径是 A → B → C → C（C重复），C 仍然是100%
   - 没有考虑渠道的真实边际贡献

**真实案例**:
某电商平台发现，搜索广告的 Last-touch 归因占比60%，但通过 Geo-experiment（关闭搜索广告），发现真实增量只有15%。原因是大部分用户本来就会主动搜索品牌词。

---

### 问题 2: Shapley Value 归因为什么"公平"？它的假设是什么？

**参考答案**:

**为什么公平**:

Shapley Value 是**唯一**满足以下4个公理的分配方案：

1. **效率性 (Efficiency)**: 
   $$\sum_{i=1}^n \phi_i(v) = v(N)$$
   所有价值被完全分配，不多不少

2. **对称性 (Symmetry)**:
   如果渠道 $i$ 和 $j$ 可互换（对所有子集贡献相同），则 $\phi_i = \phi_j$

3. **虚拟性 (Null Player)**:
   如果渠道 $i$ 对所有联盟都没有贡献，则 $\phi_i = 0$

4. **可加性 (Additivity)**:
   如果有两个价值函数 $v$ 和 $w$，则 $\phi_i(v+w) = \phi_i(v) + \phi_i(w)$

**核心思想**:
Shapley Value 计算的是渠道在**所有可能的加入顺序**中的平均边际贡献。

例如，3个渠道 A, B, C，有6种加入顺序：
```
A → B → C: A 的边际贡献 = v({A})
B → A → C: A 的边际贡献 = v({A,B}) - v({B})
C → A → B: A 的边际贡献 = v({A,C}) - v({C})
... (共6种)
```

Shapley 值 = 这6种情况的加权平均（权重由组合数决定）

**假设**:

1. **价值可分**: 总价值可以分解为各渠道的贡献之和
2. **价值函数已知**: 需要能够估计所有子集 $v(S)$ 的价值
3. **渠道间无外部性**: 除了价值函数 $v(S)$ 已经捕获的，没有其他隐藏的交互效应
4. **合作博弈**: 所有渠道"合作"产生转化（实际中可能有竞争关系）

**局限**:

1. **仍是触达归因**: 没有考虑反事实（如果没有某渠道会怎样）
2. **计算复杂度**: $O(2^n)$，渠道多时不可行
3. **数据需求**: 需要所有渠道组合的数据（实际中很多组合可能没有）
4. **静态假设**: 假设价值函数不随时间变化

**与增量归因的对比**:
- Shapley: "在已转化用户的路径中，如何公平分配功劳"
- 增量归因: "每个渠道导致了多少额外转化"（需要实验）

---

### 问题 3: 如果一个渠道在转化路径中从未出现过，但它影响了用户的线下认知，触达归因能捕捉到吗？

**参考答案**:

**不能！** 这是触达归因的根本局限。

**典型案例**:

1. **电视广告**:
   - 用户看了电视广告，产生品牌认知
   - 1周后主动搜索品牌词，点击搜索广告购买
   - 转化路径: `Search`（没有 TV）
   - 触达归因: TV 贡献 = 0%
   - 真实情况: TV 可能是最重要的渠道！

2. **线下活动**:
   - 用户参加线下品牌活动，体验产品
   - 回家后在官网购买（直接访问，无任何广告触达）
   - 转化路径: `Direct`
   - 触达归因: 线下活动贡献 = 0%

3. **口碑传播**:
   - 用户听朋友推荐产品
   - 自己搜索购买
   - 转化路径: `Search`
   - 触达归因: 口碑贡献 = 0%

**如何解决**:

**方法 1: Geo-based 实验**
```python
# 不同城市投放不同的电视广告
城市A: 投放 TV
城市B: 不投放 TV（对照组）

# 对比两组的在线转化率
uplift = (城市A转化率 - 城市B转化率) / 城市B转化率
```

**方法 2: Brand Lift Study**
```python
# 随机抽样用户做调研
实验组: 看过电视广告的用户
对照组: 未看过的用户

# 对比品牌认知度、购买意向、实际转化率
```

**方法 3: Marketing Mix Modeling (MMM)**
```R
# 回归模型
sales_t = β₀ + β₁·TV_GRP_t + β₂·Search_Spend_t + β₃·Social_Impressions_t + ε_t

# β₁ 估计的是 TV 的边际效应
# 使用时间序列数据，不需要用户级路径
```

**方法 4: Survey + 归因结合**
```python
# 在转化后问卷调查
"您是如何了解到我们的产品的？"
选项: 朋友推荐 / 电视广告 / 搜索 / 其他

# 将调研数据与触达归因结合
```

**关键洞察**:
- **触达归因只能看到"数字足迹"**
- **线下影响、口碑、品牌认知等无法追踪的因素会被系统性低估**
- **这就是为什么品牌广告团队总是抱怨归因结果不公平**

**实践建议**:
1. 触达归因 + 增量归因 结合使用
2. 为线下渠道设置单独的衡量体系（MMM, Brand Lift）
3. 不要只看归因数据做决策，结合业务常识
4. 承认归因的局限，保持谦卑

---

### 问题 4: 增量归因需要什么数据？如果无法做 RCT 怎么办？

**参考答案**:

**增量归因需要的数据**:

**理想情况（RCT）**:
```python
数据结构:
user_id | treatment | outcome | covariates
--------|-----------|---------|------------
001     | 1         | 1       | age=25, ...
002     | 0         | 0       | age=30, ...
...

# treatment: 是否暴露于该渠道（随机分配）
# outcome: 是否转化
# covariates: 协变量（用于方差缩减）
```

最低要求:
- 随机化分组（Treatment vs Control）
- 足够样本量（检验力 > 80%）
- 清晰的处理定义（什么算"暴露"）
- 无干扰假设（SUTVA）

**如果无法做 RCT，替代方案**:

**方法 1: 准实验设计**

a) **Geo-Experiment（地理实验）**
```python
# 适用于无法用户级随机化的场景（如电视广告）
处理组: 城市 A, C, E（投放广告）
对照组: 城市 B, D, F（不投放）

# 使用 DID（双重差分）估计
effect = (Y_treat_post - Y_treat_pre) - (Y_control_post - Y_control_pre)
```

优点: 不需要用户级数据  
缺点: 需要城市间可比（平行趋势假设）

b) **Regression Discontinuity（断点回归）**
```python
# 如果处理有明确的分配规则
例如：用户评分 >= 80 才发券

# 在断点附近比较
uplift = lim(x→80⁺) E[Y|X=x] - lim(x→80⁻) E[Y|X=x]
```

优点: 接近随机化  
缺点: 只能估计局部效应（断点附近）

**方法 2: 观测数据 + 因果推断方法**

a) **Propensity Score Matching（PSM）**
```python
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# 1. 估计倾向得分
ps_model = LogisticRegression()
ps_model.fit(X, treatment)
propensity_scores = ps_model.predict_proba(X)[:, 1]

# 2. 匹配
# 为每个处理组用户找到倾向得分最接近的对照组用户
matcher = NearestNeighbors(n_neighbors=1)
matcher.fit(propensity_scores[treatment==0].reshape(-1,1))
distances, indices = matcher.kneighbors(
    propensity_scores[treatment==1].reshape(-1,1)
)

# 3. 计算 ATT
treated_outcomes = outcomes[treatment==1]
matched_control_outcomes = outcomes[treatment==0][indices.flatten()]
att = (treated_outcomes - matched_control_outcomes).mean()
```

假设: 
- Unconfoundedness: 给定 X，treatment 与潜在结果独立
- Overlap: 0 < P(T=1|X) < 1

b) **Double Machine Learning (DML)**
```python
from econml.dml import DML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# 使用机器学习估计 nuisance functions
dml = DML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestClassifier(),
    discrete_treatment=True
)
dml.fit(Y, T, X=X)
treatment_effect = dml.effect(X)
```

优点: 允许高维 X，减少模型误设偏差  
缺点: 仍需要 unconfoundedness

c) **Instrumental Variable（工具变量）**
```python
# 寻找满足以下条件的工具变量 Z:
# 1. Relevance: Cov(Z, T) ≠ 0
# 2. Exclusion: Z 只通过 T 影响 Y
# 3. Exogeneity: Z 与 ε 独立

# 例如：随机广告竞价失败作为工具变量
from statsmodels.sandbox.regression.gmm import IV2SLS

iv_model = IV2SLS(Y, X, T, Z).fit()
treatment_effect = iv_model.params['T']
```

**方法 3: 自然实验**

寻找"意外"的随机化机会：
- 系统故障导致部分用户未收到广告
- 政策变化（如GDPR）影响部分地区
- 供给限制导致随机rationing

**方法 4: Holdout + 外推**
```python
# 长期设置 Holdout 组（如 5% 用户永不投放）
holdout_conversion_rate = 2.5%
full_conversion_rate = 3.2%

incremental_lift = (3.2% - 2.5%) / 2.5% = 28%

# 注意：Holdout 组可能不representative（需要定期轮换）
```

**无法做 RCT 时的决策框架**:

```
能做 Geo-Experiment? → 是 → 用 DID
        ↓
        否
        ↓
有明确分配规则? → 是 → 用 RDD
        ↓
        否
        ↓
有工具变量? → 是 → 用 IV
        ↓
        否
        ↓
观测数据 + PSM/DML（注意假设！）
        +
敏感性分析
```

**关键原则**:
1. **RCT 是金标准**，尽量创造实验机会
2. 如果用观测数据，**一定要检验假设**（平行趋势、overlap、balance）
3. **三角验证**：多种方法交叉验证
4. **诚实汇报**：说明方法局限，不要过度自信

---

### 问题 5: 一个公司同时做触达归因和增量归因，发现结果差异很大。你会相信哪个？为什么？

**参考答案**:

**短答案: 相信增量归因（如果实验设计合理）。**

**长答案: 两者回答不同问题，都有价值，但决策时应以增量归因为主。**

**详细分析**:

**案例: 搜索广告的归因差异**

```
触达归因（Last-touch）: 50% 的转化归因给搜索
增量归因（RCT 实验）: 关闭搜索广告，转化率仅下降 10%

差异: 50% vs 10%，巨大！
```

**原因分析**:

1. **Selection Bias（核心原因）**:
   ```
   用户类型分布:
   
   触达归因看到的用户（已转化）:
   - 高意向用户: 70%（这些人会主动搜索品牌词）
   - 低意向用户: 30%
   
   全量用户:
   - 高意向用户: 20%
   - 低意向用户: 80%
   
   → 搜索广告主要触达的是高意向用户（Sure Things）
   → 这些人即使不投搜索广告，也会通过直接访问购买
   → 触达归因高估了搜索的真实增量
   ```

2. **触达归因的假设**:
   - 假设：用户路径中的所有渠道都"贡献"了转化
   - 现实：很多渠道只是"参与"了，没有"导致"转化

3. **增量归因的优势**:
   - 直接回答："如果没有搜索广告，会发生什么？"
   - 有对照组（反事实）
   - 考虑了用户的 baseline conversion

**什么时候两者会接近？**

当满足以下条件时，触达归因 ≈ 增量归因：
1. **渠道间独立**: 用户要么只接触一个渠道，要么不接触
2. **无 Selection Bias**: 渠道触达是随机的（实际中几乎不可能）
3. **无 Sure Things**: 所有转化用户都是因为营销才转化

**实际建议**:

**1. 分层使用**:
```python
决策类型           |  使用哪种归因？         | 原因
-------------------|------------------------|------------------
预算分配           | 增量归因               | 需要真实 ROI
渠道效果评估       | 增量归因               | 需要因果效应
用户旅程分析       | 触达归因               | 了解路径模式
渠道协同分析       | 触达归因（Shapley）    | 分析交互效应
战略规划           | 增量归因 + MMM         | 长期影响
```

**2. 结合使用**:
```python
# 用触达归因发现异常
if Last_touch['搜索广告'] > 50%:
    # 设计 RCT 验证
    run_experiment('搜索广告关停实验')
    
    if incremental_effect < 20%:
        # 确实高估了，调整预算
        reduce_budget('搜索广告')
        increase_budget('品牌广告')
```

**3. 校准触达归因**:
```python
# 使用增量归因结果校准触达归因权重
calibration_factor = {
    '搜索广告': 增量效应 / 触达归因,  # 如 10% / 50% = 0.2
    '品牌广告': ...
}

calibrated_attribution = touch_attribution * calibration_factor
```

**警示案例**:

```
某电商平台案例:

触达归因结果:
- Retargeting: 60%
- Brand Display: 10%
- Search: 25%
- Social: 5%

基于此分配预算 → 80% 给 Retargeting

增量实验结果:
- Retargeting: 真实增量 5%（大部分是 Sure Things）
- Brand Display: 真实增量 30%（带来新客户）
- Search: 真实增量 15%
- Social: 真实增量 12%

如果只看触达归因 → 错误决策 → ROI 下降 40%
```

**判断实验是否可信的标准**:

增量归因（实验）可信，当且仅当：
1. **随机化成功**: 实验组 vs 对照组 balance（检验协变量）
2. **样本量充足**: Power ≥ 80%，MDE 合理
3. **无干扰**: SUTVA 满足（实验组不影响对照组）
4. **实验执行无误**: 无数据泄露、无早停偏差
5. **结果可复现**: 多次实验结果一致

**最终结论**:

| 维度 | 触达归因 | 增量归因 |
|------|---------|---------|
| **回答问题** | 谁参与了转化 | 谁导致了转化 |
| **因果解释** | ❌ 不可信 | ✅ 可信（如果实验设计好）|
| **决策指导** | ⚠️ 参考 | ✅ 主要依据 |
| **实施成本** | 低 | 高 |
| **适用场景** | 探索分析 | 战略决策 |

**一句话总结**:
> 触达归因帮你"看"，增量归因帮你"决策"。当两者冲突时，相信增量归因，但也要理解触达归因揭示的用户行为模式。

---

---

## 🎯 面试题模拟

### 概念题

#### 问题 1: 触达归因 vs 增量归因

**面试官**: 请解释触达归因和增量归因的区别，以及各自的应用场景。

**参考答案**:

**核心区别**：
- **触达归因 (Touch Attribution)**: 回答"谁参与了转化"
  - 数据：只看转化用户的触点路径
  - 方法：Last-touch, Shapley Value, Markov Chain
  - 问题：没有反事实，无法知道"如果没有这个渠道会怎样"

- **增量归因 (Incremental Attribution)**: 回答"谁导致了转化"
  - 数据：需要转化 + 未转化用户（实验 vs 观测）
  - 方法：RCT, Geo-experiment, PSM
  - 优势：估计因果效应（真实增量）

**应用场景**：
- **触达归因**: 快速了解用户旅程、渠道组合分析
- **增量归因**: 预算分配决策、渠道效果评估

**实际例子**：
```
场景：Retargeting 广告

触达归因结果：
- Last-touch 归因 60%（因为总是最后触点）
- 看起来效果很好

增量归因结果（通过 Ghost Ads 实验）：
- 真实增量只有 +2%
- 大部分用户本来就会转化（Sure Things）

结论：触达归因高估了 Retargeting 的价值
```

---

#### 问题 2: Shapley Value 的优势和局限

**面试官**: 为什么说 Shapley Value 归因比规则归因更"公平"？它有什么局限性？

**参考答案**:

**Shapley Value 的优势**：
1. **公理化基础**: 唯一满足 4 个公理的分配方案
   - 效率性：Σφᵢ = v(N)
   - 对称性：可互换玩家贡献相等
   - 虚拟性：无贡献玩家得 0
   - 可加性：线性

2. **考虑所有可能的加入顺序**: 不像 Last-touch 那样武断

3. **理论保证**: 博弈论证明的"公平分配"

**局限性**：
1. **计算复杂度**: O(2ⁿ)，渠道多时不可行
   - 5 个渠道：32 种子集
   - 10 个渠道：1024 种子集

2. **数据需求**: 需要估计所有子集的价值函数 v(S)
   - 实际中很多组合没有数据

3. **仍是触达归因**: 没有回答"如果没有某渠道会怎样"
   - 只是更公平地分配"功劳"
   - 不等于因果效应

4. **假设独立性**: 渠道间可能有协同/替代效应

**工程实践**：
- Google 的 Data-Driven Attribution 使用近似算法
- 限制渠道数（聚合小渠道）
- 使用采样估计

---

### 编程题

#### 问题 3: 实现简化版 Shapley Value

**面试官**: 手写一个简化版的 Shapley Value 计算函数（假设只有 3 个渠道）。

**参考答案**:

```python
def shapley_value_simple(conversion_data, channels):
    """
    简化版 Shapley Value 计算
    
    Args:
        conversion_data: List[Tuple[Set[str], float]]
            [(渠道集合, 转化价值), ...]
        channels: List[str]
            渠道列表
    
    Returns:
        Dict[str, float]: 各渠道的 Shapley 值
    """
    from itertools import combinations
    
    # 构建价值函数
    def v(channel_set):
        \"\"\"计算包含 channel_set 中所有渠道的路径的平均价值\"\"\"
        matching_values = [
            value for path_channels, value in conversion_data
            if channel_set.issubset(path_channels)
        ]
        return np.mean(matching_values) if matching_values else 0.0
    
    shapley = {}
    n = len(channels)
    
    for channel in channels:
        channel_value = 0.0
        other_channels = [c for c in channels if c != channel]
        
        # 遍历所有不包含当前渠道的子集
        for size in range(n):
            for subset in combinations(other_channels, size):
                subset_set = set(subset)
                
                # 边际贡献
                marginal = v(subset_set | {channel}) - v(subset_set)
                
                # Shapley 权重
                from math import factorial
                weight = factorial(size) * factorial(n - size - 1) / factorial(n)
                
                channel_value += weight * marginal
        
        shapley[channel] = channel_value
    
    return shapley

# 测试
data = [
    ({'Search'}, 100),
    ({'Social', 'Search'}, 150),
    ({'Email', 'Search'}, 120),
    ({'Social', 'Email', 'Search'}, 200),
]
channels = ['Search', 'Social', 'Email']
result = shapley_value_simple(data, channels)
print("Shapley Values:", result)
```

**关键点**：
1. 理解 Shapley 公式的组合优化本质
2. 正确计算权重（排列组合）
3. 处理边界情况（空集、全集）

---

#### 问题 4: Markov Chain 归因的转移矩阵

**面试官**: 给定用户路径数据，如何构建 Markov Chain 的转移矩阵？

**参考答案**:

```python
def build_transition_matrix(paths):
    """
    构建 Markov Chain 转移矩阵
    
    Args:
        paths: List[List[str]]
            用户路径列表
    
    Returns:
        pd.DataFrame: 转移概率矩阵
    """
    from collections import defaultdict
    
    # 统计转移次数
    transitions = defaultdict(lambda: defaultdict(int))
    
    for path in paths:
        # Start -> 第一个触点
        transitions['Start'][path[0]] += 1
        
        # 触点之间的转移
        for i in range(len(path) - 1):
            transitions[path[i]][path[i+1]] += 1
        
        # 最后一个触点 -> Conversion
        transitions[path[-1]]['Conversion'] += 1
    
    # 转换为概率（归一化）
    transition_probs = {}
    for from_state, to_states in transitions.items():
        total = sum(to_states.values())
        transition_probs[from_state] = {
            to_state: count / total
            for to_state, count in to_states.items()
        }
    
    # 转换为 DataFrame
    all_states = set(['Start', 'Conversion'])
    for path in paths:
        all_states.update(path)
    
    matrix = pd.DataFrame(0.0, index=all_states, columns=all_states)
    for from_state, to_dict in transition_probs.items():
        for to_state, prob in to_dict.items():
            matrix.loc[from_state, to_state] = prob
    
    return matrix

# 测试
paths = [
    ['Social', 'Search'],
    ['Search'],
    ['Social', 'Email', 'Search'],
    ['Display', 'Search'],
]
matrix = build_transition_matrix(paths)
print("转移矩阵:")
print(matrix)
```

**关键点**：
1. 理解马尔可夫链的状态转移
2. 正确归一化（每行和为 1）
3. 处理 Start 和 Conversion 特殊状态

---

### 场景题

#### 问题 5: 实际业务问题

**面试官**: 你们公司的 Last-touch 归因显示搜索广告贡献了 50% 的转化，但 CMO 怀疑这个数字。如何验证？

**参考答案**:

**分析思路**：

1. **识别问题**：
   - Last-touch 倾向于高估"最后触点"
   - 搜索广告经常出现在路径末端（用户主动搜索）
   - 可能是「顺水推舟」而非「雪中送炭」

2. **验证方法**：

   **方法 1: 对比多种归因方法**
   ```
   - Last-touch: 50%
   - First-touch: 20%
   - Linear: 35%
   - Shapley: 30%
   
   → 说明 Last-touch 确实高估了
   ```

   **方法 2: 分析用户路径**
   ```python
   # 检查搜索广告出现的位置
   search_positions = []
   for path in paths:
       if 'Search' in path:
           pos = path.index('Search') / len(path)
           search_positions.append(pos)
   
   # 如果平均位置 > 0.7，说明经常在末尾
   avg_position = np.mean(search_positions)
   print(f"搜索广告平均位置: {avg_position:.2f} (1.0=最后)")
   ```

   **方法 3: 增量实验（推荐）**
   ```
   设计 Geo-experiment:
   - 选择相似的地区
   - 实验组：正常投放搜索广告
   - 对照组：关闭搜索广告
   - 对比转化率差异
   
   如果实际增量只有 10%，说明 Last-touch 的 50% 严重高估
   ```

3. **建议方案**：
   - 短期：改用 Shapley 或 Position-based 归因
   - 中期：实施增量实验
   - 长期：建立 MMM（Marketing Mix Modeling）

**面试加分点**：
- 提到具体的实验设计
- 讨论数据偏差（Cookie 丢失、跨设备）
- 考虑业务影响（预算重分配）

---

### 思维拓展题

#### 问题 6: 线下渠道的归因

**面试官**: 电视广告、线下活动这些渠道无法精准追踪，如何做归因？

**参考答案**:

**挑战**：
- 无法追踪个体级触达
- 无法记录在转化路径中
- 触达归因失效

**解决方案**：

1. **Geo-based 实验**：
   ```
   - 不同城市投放不同的电视广告
   - 对比各城市的在线转化率
   - 识别电视广告的增量效应
   ```

2. **时间序列分析**：
   ```python
   # 检查电视广告播出时段前后的搜索量变化
   tv_schedule = [...]  # 广告播出时间
   search_volume = [...]  # 搜索量
   
   # DID 分析
   before = search_volume[tv前]
   after = search_volume[tv后]
   uplift = (after - before) / before
   ```

3. **Brand Lift Study**：
   ```
   - 随机选择用户做品牌认知调研
   - 实验组：看过电视广告
   - 对照组：未看过
   - 对比后续转化率
   ```

4. **MMM (Marketing Mix Modeling)**：
   ```R
   # 使用回归模型
   sales ~ TV_GRP + Search_Spend + Social_Impressions + ...
   
   # 估计各渠道的弹性系数
   # TV_GRP 的系数 = 电视广告的边际效应
   ```

**关键洞察**：
- 线下渠道更适合增量归因
- 需要宏观层面的因果推断（Geo, MMM）
- 不能只看触达归因

---

---

## 📚 总结

### 核心知识点

| 概念 | 要点 | 适用场景 |
|------|------|----------|
| **触达归因** | 基于转化路径分配"功劳" | 了解用户旅程、渠道组合 |
| **Last-touch** | 100% 给最后触点 | 短周期、直接转化 |
| **Shapley Value** | 基于合作博弈论，满足公理性 | 需要"公平"分配 |
| **Markov Chain** | 基于转移概率，移除效应 | 考虑路径动态 |
| **增量归因** | 基于反事实，估计因果效应 | **预算决策、效果评估** |

### 关键公式

**Shapley Value:**
$$
\phi_i(v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|! \cdot (|N| - |S| - 1)!}{|N|!} \cdot [v(S \cup \{i\}) - v(S)]
$$

**Removal Effect (Markov):**
$$
\text{RE}_c = \frac{P(\text{Conv}) - P(\text{Conv | remove } c)}{P(\text{Conv})}
$$

### 最重要的启示

> **"参与了转化" ≠ "导致了转化"**
>
> 触达归因告诉你用户经历了什么，  
> 增量归因告诉你什么真正起了作用。
>
> 用触达归因理解，用增量归因决策。

### 下一步

- 在 `part6_2_coupon_optimization.ipynb` 中，我们会学习如何用因果推断方法做优惠券优化
- 在 `part6_4_budget_allocation.ipynb` 中，我们会学习如何结合归因结果进行预算优化

---

**恭喜你完成了营销归因的学习！** 🎉